In [ ]:
import numpy as np
import pandas as pd

from arch import arch_model
from arch.univariate import EGARCH
import matplotlib.pyplot as plt
import statsmodels.api as sm

# local module
from data_cleaning.import_data import import_data
# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

In [120]:
# estimate and report parameter estimates and misspec tests for all pairs
pairs = {'USDRUB':'2005-03-03', 'USDZAR':'2003-02-23', 'USDBRL':'2003-10-01', 'EURUSD':'2000-03-15', 'USDJPY':'2000-02-23', 'GBPUSD':'2000-02-23'}
params = pd.DataFrame(index=['omega', 'alpha', 'beta', 'nu', 'aic', 'logL'])
tvals = pd.DataFrame(index=['omega', 'alpha', 'beta', 'nu'])
ljung_box = pd.DataFrame()
ljung_box.index += 1 
noarch = pd.DataFrame(index=['LM statistic', 'Degrees of Freedom', 'P-value'])

for pair in pairs:
    df = import_data(fx_pair=pair)
    ret = df['log_ret']
    ret = ret.loc[pairs[pair]:'2020-12-31'] * 100
    #ret[0] = 0

    # fit model and save residuals
    garch = arch_model(ret, mean='Zero', vol='GARCH', p=1,q=1, dist = 'Studentst')
    garch_fit = garch.fit(update_freq = 0, disp = 'off')
    res = garch_fit.resid
    
    # fetch params
    params_it = []
    params_it.append(garch_fit.params['omega'])
    params_it.append(garch_fit.params['alpha[1]'])
    params_it.append(garch_fit.params['beta[1]'])
    params_it.append(garch_fit.params['nu'])
    params_it.append(garch_fit.aic)
    params_it.append(garch_fit.loglikelihood)
    params[pair] = params_it

    # fetch t-values
    tvals_it = []
    tvals_it.append(garch_fit.tvalues['omega'])
    tvals_it.append(garch_fit.tvalues['alpha[1]'])
    tvals_it.append(garch_fit.tvalues['beta[1]'])
    tvals_it.append(garch_fit.tvalues['nu'])
    tvals[pair] = tvals_it

    # misspecification tests
    lb = sm.stats.acorr_ljungbox(res, lags=list(range(1,11)), return_df=True) # Ljung-Box test. H0: No residual autocorrelation
    ljung_box[pair] = lb['lb_pvalue']

    noarch_test = garch_fit.arch_lm_test(2, True)
    noarch_it = []
    noarch_it.append(noarch_test.stat)
    noarch_it.append(noarch_test.df)
    noarch_it.append(noarch_test.pval)
    noarch[pair] = noarch_it

In [121]:
params

,USDRUB,USDZAR,USDBRL,EURUSD,USDJPY,GBPUSD
omega,0.000894,0.019531,0.009948,0.000738,0.003344,0.002617
alpha,0.069400,0.059770,0.103900,0.035926,0.057460,0.041727
beta,0.930601,0.923261,0.892023,0.962593,0.935844,0.950618
nu,5.731351,10.306982,7.680272,10.001919,5.444492,8.650045
aic,7138.428005,13265.591679,11448.033589,8909.995521,8939.047689,8528.848334
logL,-3565.214002,-6628.795839,-5720.016795,-4450.997761,-4465.523845,-4260.424167


In [122]:
tvals

,USDRUB,USDZAR,USDBRL,EURUSD,USDJPY,GBPUSD
omega,2.612845,3.483740,3.130723,2.016822,2.932277,2.611009
alpha,7.361576,6.607776,8.356767,9.354033,7.223266,6.184578
beta,96.456443,76.388754,72.100471,245.853935,102.809454,115.229650
nu,12.292271,6.606436,8.449930,7.917417,13.637583,7.744312


In [123]:
lb

,lb_stat,lb_pvalue
1,6.074063,0.013718
2,6.079533,0.047846
3,6.087774,0.107417
4,6.091191,0.192441
5,11.567040,0.041227
6,12.001180,0.061942
7,12.029886,0.099580
8,12.792211,0.119202
9,14.191104,0.115688
10,18.296667,0.050161


In [124]:
noarch

,USDRUB,USDZAR,USDBRL,EURUSD,USDJPY,GBPUSD
LM statistic,17.489771,7.281694,6.270516,4.256475,0.15428,2.940036
Degrees of Freedom,2.000000,2.000000,2.000000,2.000000,2.00000,2.000000
P-value,0.000159,0.026230,0.043489,0.119047,0.92576,0.229921
